In [9]:
import skweak.utils
from spacy import displacy
import os
import myutils as utils
import numpy as np

In [2]:
data_path = 'results/train_corpus_IO.spacy'
docs = skweak.utils.docbin_reader(data_path, spacy_model_name='en_core_web_lg')
data = utils.load_json('data/train-soft.json')

In [3]:
docs = list(docs)

In [4]:
len(docs)

2407

In [38]:
sample_size = 10
i = 0
for doc, d in zip(docs, data):
    if i < sample_size:
        skweak.utils.display_entities(doc, 'hmm')
        print(d['label'])
    i+=1

PARTIAL_CORRECT


CORRECT


CORRECT


CORRECT


CORRECT


PARTIAL_CORRECT


PARTIAL_CORRECT


PARTIAL_CORRECT


PARTIAL_CORRECT


PARTIAL_CORRECT


In [7]:
def relation(spans, doc):
    spans = [len(s) for s in spans]
    return sum(spans)/len(doc)

In [12]:
disable_lang_filter = False
for language in ['de', 'en', 'combined']:
    relations_correct, relations_partial, relations_incorrect = [], [], []
    len_rubrics_correct, len_rubrics_partial, len_rubrics_incorrect = [], [], []
    len_rubrics_average_correct, len_rubrics_average_partial, len_rubrics_average_incorrect = [], [], []
    if language == 'combined':
        disable_lang_filter = True
    for doc, d in zip(docs, data):
        c = d['label']
        lang = d['lang']
        spans = skweak.utils.get_spans(doc, ["hmm"])
        if lang == language or disable_lang_filter:
            rel = relation(spans, doc)
            len_rubrics = [len(s) for s in spans]
            if len(len_rubrics) > 0:
                len_rubrics_average = np.average(len_rubrics)
            else:
                len_rubrics_average = 0
            if c == 'CORRECT':
                relations_correct.append(rel)
                len_rubrics_correct.append(len(len_rubrics))
                len_rubrics_average_correct.append(len_rubrics_average)
            elif c == 'PARTIAL_CORRECT':
                relations_partial.append(rel)
                len_rubrics_partial.append(len(len_rubrics))
                len_rubrics_average_partial.append(len_rubrics_average)
            elif c == 'INCORRECT':
                relations_incorrect.append(rel)
                len_rubrics_incorrect.append(len(len_rubrics))
                len_rubrics_average_incorrect.append(len_rubrics_average)
    print(language.upper())
    print('AVERAGE NUMBER OF KEY ELEMENTS')
    print(np.average(len_rubrics_correct), np.average(len_rubrics_partial), np.average(len_rubrics_incorrect))
    print('AVERAGE NUMBER OF TOKENS PER KEY ELEMENT')
    print(np.average(len_rubrics_average_correct), np.average(len_rubrics_average_partial), np.average(len_rubrics_average_incorrect))
    print('AVERAGE RELATION OF I-CUE TO OUT')
    print(np.average(relations_correct), np.average(relations_partial), np.average(relations_incorrect))
    print(10*'-')

DE
AVERAGE NUMBER OF KEY ELEMENTS
1.4036036036036037 1.1666666666666667 0.9404761904761905
AVERAGE NUMBER OF TOKENS PER KEY ELEMENT
9.06 7.833333333333333 5.259920634920634
AVERAGE RELATION OF I-CUE TO OUT
0.7820578182741155 0.7791386483633941 0.6948670832209384
----------
EN
AVERAGE NUMBER OF KEY ELEMENTS
2.783661119515885 2.963963963963964 0.8229166666666666
AVERAGE NUMBER OF TOKENS PER KEY ELEMENT
22.21207962486329 20.18655321988655 5.722371031746032
AVERAGE RELATION OF I-CUE TO OUT
0.6381925669334025 0.6096819641151562 0.33625420539665996
----------
COMBINED
AVERAGE NUMBER OF KEY ELEMENTS
2.153782894736842 1.8868832731648617 0.8777777777777778
AVERAGE NUMBER OF TOKENS PER KEY ELEMENT
16.20928012502848 12.783540580291485 5.506560846560847
AVERAGE RELATION OF I-CUE TO OUT
0.7038547499055208 0.7112336232675298 0.5036068817146565
----------


In [29]:
labels = []
for doc, d in zip(docs, data):
    label = np.zeros((len(doc)))
    probs = doc.spans["hmm"].attrs['probs']
    for k,v in probs.items():
        label[int(k)] = v['I-CUE']
    labels.append(label.tolist())

In [37]:
for t,l in zip(docs[100], labels[100]):
    print(t,l)

Ich 0.9999121144286632
zeige 0.9999819354349667
mein 0.9999824318914303
informationsschreiben 1.0


In [35]:
data[100]

{'lang': 'de',
 'question_id': '1',
 'question': 'Frage 1: Wie reagierst du, wenn du auf deine Tätigkeit angesprochen wirst?',
 'reference_answer': 'Der Jobber soll sich in diesem Fall dem Personal gegenüber zu erkennen geben (0.25 P) und das entsprechende Informationsschreiben in der App vorzeigen (0.25 P). Zusätzlich muss notiert werden, zu welchem Zeitpunkt (0.25 P) des Jobs der Jobber enttarnt wurde. Zentrale Frage ist dabei, ob ein neutrales, unvoreingenommenes Verkaufsgespräch stattgefunden hat. Der Job soll mit Erlaubnis der Mitarbeiter bis zum Ende durchgeführt (0.25 P) werden.',
 'score': 0.25,
 'label': 'PARTIAL_CORRECT',
 'student_answer': 'Ich zeige mein informationsschreiben',
 'labeling_functions': {'LF_noun_phrases': [0.0, 0.0, 0.0, 0.0],
  'LF_lemma_match': [0.0, 0.0, 0.0, 0.0],
  'LF_pos_match': [0.0, 0.0, 0.0, 0.0],
  'LF_lemma_match_without_stopwords': [0.0, 0.0, 0.0, 0.0],
  'LF_pos_match_without_stopwords': [0.0, 1.0, 1.0, 0.0],
  'LF_shape_match': [0.0, 0.0, 0.0, 

In [39]:
sample_size = 10
i = 0
for doc, d, l in zip(docs, data, labels):
    if i < sample_size:
        skweak.utils.display_entities(doc, 'hmm')
        print(l)
        print(d['label'])
    i+=1

[0.6783796984540141, 0.6291256290976186, 0.5850250739548654, 0.545424470259314, 0.509736943764081, 0.4774336113641682, 0.44803574313836547, 0.42110766765314567, 0.39625031538576977, 0.37309530458085904, 0.3512994818958464, 0.3305398369258068, 0.3105087152444384, 0.2909092590173608, 0.2714510076192552, 0.25184559305625864, 0.231802466401457, 0.21102459190877979, 0.18920404499343016, 0.16601744884175135, 0.14112118202158558, 0.11414628607059749, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6360403616122355, 0.8036668348974945, 0.8132866313597135, 0.7104784675677673, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.848724541556727, 0.9761042426066322, 0.9953779103424634, 0.9978644247853713, 0.9953449226233314, 0.9758819709410539, 0.8472598599985052, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8475042555417195, 0.97617017161929, 0.9956839061082632, 0.998501778325434, 0.9979749022588701, 0.991606932

[0.9998637844349085, 0.9999798397309179, 0.9999727117628431, 0.9999936679976386, 0.9999905426128917, 0.999972619715912, 0.9999724560375514, 0.9999717790447015, 0.9998950114040581, 0.9911720573049806, 0.0, 0.9911720573451099, 0.9998950114587641, 0.9999717807157655, 0.9999726413715676, 0.9999936743089092, 0.9999936925754549, 0.9999816345586859, 0.9999605900409969, 0.9963974969144082, 0.3911019142515997, 0.9963974964354859, 0.9999605891457057, 0.9999815634066009, 0.9999816862562872, 0.9999815823332064, 0.9999638044943377, 0.9912225201091761, 0.0]
CORRECT


[0.9997638243777301, 0.9999576080198113, 0.9999600589325544, 0.999959989649165, 0.9999520627407832, 0.9999519425583334, 0.9999519407361915, 0.999951940708567, 0.9999519407081406, 0.9999519407081691, 0.9999519407093912, 0.9999519407911849, 0.9999519461858493, 0.9999523020032721, 0.9999757707540133, 0.9999758315165376, 0.9999602530178114, 0.9999570915072008, 0.9997227916668668, 0.9847919182272886, 0.0, 0.15448353999001263, 0.44399776559159365, 0.9940029527594669, 0.9999023413102313, 0.9999656191446377, 0.9999663575205935, 0.9999719270470989, 0.999971976461658, 0.999971927112865, 0.9999663648661078, 0.9999663051878703, 0.9999663029378563, 0.9999661528219581, 0.9999521580240651, 0.9999520659867279, 0.9999599888580216, 0.9999599933202503, 0.9999524231328167, 0.999975773515485, 0.9999760672185567, 0.9999908553491579, 0.999973624869488, 0.9999671977437116, 0.9998657260018384, 0.989865206144759, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10764716945579399, 0.9857491821202613, 0.9997234901994588, 0.9999458804041453, 0.9999494195706408, 0.9999494758937097, 0.9999494767900283, 0.9999494768042952, 0.9999494768045226, 0.999949476805489, 0.9999494768647736, 0.9999494805904441, 0.9999497146999173, 0.9999644254105062, 0.9999645791110189, 0.9999635218178966, 0.9998694473733029, 0.9915137476328928, 0.24935998943992835, 0.9915145583992838, 0.9998705676811281, 0.9999913283303642, 0.9998778223242856, 0.986795419106859, 0.9999952063726099, 0.9999990930798068, 0.9999990942242344, 0.999999094207693, 0.9999990367197803, 0.9998037999800453, 0.33675322730049484, 0.9998037999970382, 0.9999990370034286, 1.0, 0.9999990944912845, 0.9999990942246891, 0.9999990942245754, 0.9999990942246891, 0.9999990944912845, 1.0, 0.9999990947579368, 1.0, 0.9999990944912845, 0.999999094224746, 0.9999990944912845, 1.0, 0.33520642925657185, 0.9949948958185513, 0.9999788369861216, 0.9999764403752579, 0.9999669025915404, 0.9999884

[0.9998498779645272, 1.0, 0.9999927597178616, 0.9999880064886371, 0.9999701257000276, 0.9966151833155192, 0.2701969588081753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
CORRECT


[0.9998637844349227, 0.9999798397312873, 0.9999727118195144, 0.9999936744528074, 0.999993692828165, 0.9999817585824888, 0.9999817402540608, 0.9999905607718761, 0.9999905331137642, 0.9999726231218196, 0.9999728548362501, 0.9942138855904814, 0.3418628239185951, 0.21699802531015006, 0.13729239460152773, 0.0, 0.0, 0.0, 0.0, 0.0]
PARTIAL_CORRECT


[0.9999530508846491, 0.9999904734713876, 0.999990532836101, 0.9999726196823753, 0.9999724620350632, 0.9999724606476672, 0.9999724606410167, 0.9999724612725865, 0.9999725330485028, 0.9999806888022421, 1.0, 0.9999871435078327, 0.9999870899679336, 0.9999870897449398, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897427797, 0.9999870894441282, 0.9999870177261699, 0.9999697961598366, 0.99583439570739, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9958344701530383, 0.9999697964698513, 0.9999870177274773, 0.9999870894441282, 0.9999870897427797, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897440303, 0.9999870897433482, 0.9999870895866328, 0.9999870519

[0.99986449203515, 0.9999719760136485, 1.0, 0.9999729126737841, 0.9999726760861698, 0.9999726721071432, 0.9999724530620828, 0.9999473745080786, 0.9912765955030386, 0.0]
PARTIAL_CORRECT


[0.9998837476826398, 0.9999759635835527, 0.9999859412605501, 0.9999767726555518, 0.9999919291286529, 0.9999858218472018, 0.9999767431911378, 0.9999881087633918, 0.9999768474692631, 0.9999997744886683, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
PARTIAL_CORRECT


[0.9999086652951362, 0.9999810987706326, 0.9999638434016109, 0.9999815105406396, 0.9999780361993301, 0.9999870556620664, 0.9999964082182645, 0.9999830588650142, 0.9999970118728094, 0.9999970253757529, 0.9999970548353927, 0.9999858118842834, 0.9999857474942816, 0.9999829599846681, 0.9999896351246762, 0.9999941083392132, 0.9999857984212244, 0.999985760458887, 0.9999857984603889, 0.9999941168727813, 0.9999941115731867, 0.9999830056712201, 0.9999895976378539, 0.9999828846500074, 0.9999720549909172, 0.9980092217069735, 0.409478747616983, 0.9999758648185586, 0.9999940979867803, 0.9999941115372051, 0.9999830056720728, 0.9999895978378269, 0.9999829442847834, 0.9999829234610639, 0.9999857898834074, 0.9999950765213285, 0.999985789882441, 0.9999829232581933, 0.9999829073584511, 0.999982882360621, 0.9999783520076005, 0.9966297487151765, 0.9999645471766652, 0.9999896773153011, 0.9999829813434463, 0.9999895980668467, 0.9999830368995547, 0.9999998026396691, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0